In [1]:
import os
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/panaskar.sanket/datascience_project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="panaskar.sanket"
os.environ['MLFLOW_TRACKING_PASSWORD']="7934302e1edc9497a6be87caa910a095bce8f64f"

In [2]:
%pwd

'c:\\Users\\Sanket Panaskar\\Desktop\\Projects\\mlops\\datascience_project\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Sanket Panaskar\\Desktop\\Projects\\mlops\\datascience_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json


In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                       params_filepath=PARAMS_FILE_PATH,
                       schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/panaskar.sanket/datascience_project.mlflow"
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [8]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config

    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis =1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme



        with mlflow.start_run():
            predicted_values = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metric(test_y, predicted_values)
            
            # Saving metrics as json in local.
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Logging mlflow parameters

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [9]:
try:
    config=ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-11 11:01:11,248: INFO: common: yaml file config\config.yaml loaded successfully.]
[2025-05-11 11:01:11,250: INFO: common: yaml file params.yaml loaded successfully.]
[2025-05-11 11:01:11,252: INFO: common: yaml file schema.yaml loaded successfully.]
[2025-05-11 11:01:11,253: INFO: common: created directory at: artifacts]
[2025-05-11 11:01:11,254: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-11 11:01:11,679: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]


2025/05/11 11:01:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/05/11 11:01:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run rambunctious-frog-32 at: https://dagshub.com/panaskar.sanket/datascience_project.mlflow/#/experiments/0/runs/d8fb02193829481f8e80b45018bd878c
🧪 View experiment at: https://dagshub.com/panaskar.sanket/datascience_project.mlflow/#/experiments/0
